In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools

In [3]:
# import the required libraries
from czitools import metadata_tools as czimd
from czitools import read_tools as czird
from czitools import misc_tools
from IPython.display import display, HTML
from pathlib import Path
import os
import dask.array as da
import ipywidgets as widgets

if not IN_COLAB:
    from czitools import napari_tools
    import napari


In [4]:
czifiles = [("CellDivision_T=10_Z=15_CH=2_DCV_small.czi", r"https://github.com/sebi06/czitools/raw/main/data/CellDivision_T%3D10_Z%3D15_CH%3D2_DCV_small.czi"),
            ("CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi", r"https://github.com/sebi06/czitools/raw/main/data/CellDivision_T%3D3_Z%3D5_CH%3D2_X%3D240_Y%3D170.czi"),
            ("WP96_4Pos_B4-10_DAPI.czi", r"https://github.com/sebi06/czitools/raw/main/data/WP96_4Pos_B4-10_DAPI.czi"),
            ("w96_A1+A2.czi", r"https://github.com/sebi06/czitools/raw/main/data/w96_A1%2BA2.czi")]

In [5]:
wd = widgets.Select(
    options=czifiles,
    value=r"https://github.com/sebi06/czitools/raw/main/data/CellDivision_T%3D3_Z%3D5_CH%3D2_X%3D240_Y%3D170.czi",
    description='CZI Files:',
    layout={'width': 'max-content'}
)

display(wd)

Select(description='CZI Files:', index=1, layout=Layout(width='max-content'), options=(('CellDivision_T=10_Z=1…

In [6]:
selected_file = wd.value

print(f"Selected CZI File {selected_file}")

Selected CZI File https://github.com/sebi06/czitools/raw/main/data/CellDivision_T%3D3_Z%3D5_CH%3D2_X%3D240_Y%3D170.czi


In [7]:
# get info about attachments only
attachments = czimd.CziAttachments(selected_file)

for k, v in attachments.__dict__.items():
    print(k, v)

2024-02-10 14:01:44,182 -  INFO - Reading AttachmentImages from CZI image data.
2024-02-10 14:01:44,303 -  WARNING - Reading Attachments from CZI via a link is not supported.


czisource https://github.com/sebi06/czitools/raw/main/data/CellDivision_T%3D3_Z%3D5_CH%3D2_X%3D240_Y%3D170.czi
names []


In [16]:
#%%capture
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(selected_file)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_md_dict_red(mdata, sort=False, remove_none=True)

# convert metadata dictionary to a pandas dataframe
mdframe = misc_tools.md2dataframe(mdict)

# and display it nicely as a HTML inside the jupyter notebook
#display(HTML(mdframe.to_html()))

a1 = widgets.Output(layout={"scroll": "auto", "height": "300px"})
with a1:
    display(mdframe)
display(a1)

2024-02-10 14:10:01,825 -  INFO - FilePath is a valid link. Only pylibCZIrw functionality is available.
2024-02-10 14:10:02,462 -  INFO - Reading Dimensions from CZI image data.
2024-02-10 14:10:02,733 -  INFO - Reading BoundingBoxes from CZI image data.
2024-02-10 14:10:02,734 -  INFO - FilePath is a valid link. Only pylibCZIrw functionality is available.
2024-02-10 14:10:02,982 -  INFO - Reading Channel Information from CZI image data.
2024-02-10 14:10:02,985 -  INFO - Reading Scaling from CZI image data.
2024-02-10 14:10:02,987 -  INFO - Reading Objective Information from CZI image data.
2024-02-10 14:10:02,989 -  INFO - Reading Detector Information from CZI image data.
2024-02-10 14:10:02,994 -  INFO - Reading Microscope Information from CZI image data.
2024-02-10 14:10:02,996 -  INFO - Reading SampleCarrier Information from CZI image data.
2024-02-10 14:10:02,997 -  INFO - Reading Dimensions from CZI image data.
2024-02-10 14:10:02,999 -  INFO - No Scene or Well information found.

[WinError 123] Die Syntax für den Dateinamen, Verzeichnisnamen oder die Datenträgerbezeichnung ist falsch: 'e:\\Github\\czitools\\demo\\notebooks\\https:\\github.com\\sebi06\\czitools\\raw\\main\\data\\CellDivision_T%3D3_Z%3D5_CH%3D2_X%3D240_Y%3D170.czi'


Output(layout=Layout(height='300px'))

In [ ]:
# return array with dimension order STCZYX(A)
array6d, mdata= czird.read_6darray(selected_file, use_dask=True, chunk_zyx=True)

# show dask array structure
if isinstance(array6d, da.Array):
    print(array6d)
else:
    print("Shape:", array6d.shape, "dtype:", array6d.dtype)


In [ ]:
if not IN_COLAB:

    # show array inside napari viewer
    viewer = napari.Viewer()
    layers = napari_tools.show(viewer, array6d, mdata,
                               blending="additive",
                               contrast='from_czi',
                               gamma=0.85,
                               show_metadata="table",
                               name_sliders=True)

In [ ]:
napari.utils.nbscreenshot(viewer)